#04_DSMarket_SS_calculation

## 1. Import

In [142]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.stats import norm

In [143]:
df_sales = pd.read_parquet("df_forecasting.parquet", engine="pyarrow")
print(df_sales.shape)
df_sales.head(3)

(8354260, 15)


,id,Unnamed: 0,item,category,department,store,store_code,region,yearweek,n_sales,revenue,avg_sell_price,event,cluster_store,cluster_item
0,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201104,0,0.0,NaN,Without event,1,0
1,ACCESORIES_1_001_BOS_1,1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201105,0,0.0,NaN,SuperBowl,1,0
2,ACCESORIES_1_001_BOS_1,2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201106,0,0.0,NaN,Without event,1,0


In [144]:
df_prediction = pd.read_parquet("rmse_por_id.parquet", engine="pyarrow")
print(df_prediction.shape)
df_prediction.head(3)

(30490, 2)


,id,rmse_mean
0,ACCESORIES_1_001_BOS_1,1.006209
1,ACCESORIES_1_001_BOS_2,1.031386
2,ACCESORIES_1_001_BOS_3,1.477416


In [145]:
df_prediction.head(3)

,id,rmse_mean
0,ACCESORIES_1_001_BOS_1,1.006209
1,ACCESORIES_1_001_BOS_2,1.031386
2,ACCESORIES_1_001_BOS_3,1.477416


In [146]:
df_sales.head(3)

,id,Unnamed: 0,item,category,department,store,store_code,region,yearweek,n_sales,revenue,avg_sell_price,event,cluster_store,cluster_item
0,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201104,0,0.0,NaN,Without event,1,0
1,ACCESORIES_1_001_BOS_1,1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201105,0,0.0,NaN,SuperBowl,1,0
2,ACCESORIES_1_001_BOS_1,2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201106,0,0.0,NaN,Without event,1,0


## 2. Cálculo del stock de seguridad

Para calcular el **stock de seguridad**, utilizaremos el **RMSE** obtenido para cada item y tienda durante la fase de predicción.
A partir de este valor, calcularemos el stock de seguridad del producto en cada tienda aplicando la siguiente fórmula:

**SS** = Z ×RMSE ×√L
​
- **Z** es el nivel de servicio deseado, que representa la probabilidad de no tener una rotura de stock durante el tiempo de reposición.
Cuanto mayor sea el nivel de servicio (por ejemplo, 95% o 99%), mayor será el valor de Z y, por tanto, el nivel de protección frente a la variabilidad de la demanda.

- **L** es el lead time o tiempo de aprovisionamiento, es decir, el número de periodos (en este caso semanas) que transcurren desde que se realiza el pedido hasta que la mercancía llega al almacén o tienda.

De esta forma, el stock de seguridad actúa como una reserva adicional que permite absorber la incertidumbre de la demanda y garantizar la disponibilidad del producto incluso en situaciones imprevistas.

In [147]:
service_level = 0.95
z = norm.ppf(service_level)
L = 1 #lead time of one week

In [148]:
df_prediction["SS"] = z*df_prediction["rmse_mean"] * np.sqrt(L)

In [149]:
df_prediction

,id,rmse_mean,SS
0,ACCESORIES_1_001_BOS_1,1.006209,1.655066
1,ACCESORIES_1_001_BOS_2,1.031386,1.696480
2,ACCESORIES_1_001_BOS_3,1.477416,2.430132
3,ACCESORIES_1_001_NYC_1,2.073110,3.409963
4,ACCESORIES_1_001_NYC_2,2.703180,4.446336
...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,5.298480,8.715224
30486,SUPERMARKET_3_827_NYC_4,4.934147,8.115950
30487,SUPERMARKET_3_827_PHI_1,9.285024,15.272505
30488,SUPERMARKET_3_827_PHI_2,7.789934,12.813302


## 2. ROP — nivel de pedido (Reorder Point)

El punto de pedido (ROP) indica el nivel de inventario en el que se debe realizar un nuevo pedido para evitar una rotura de stock, y se calcula de la siguiente manera:

𝑅𝑂𝑃 =𝐷𝐿 +𝑆𝑆

donde:

**DL** = demanda media durante el lead time, que en nuestro caso utilizaremos como la demanda media del artículo en la tienda

**SS** = stock de seguridad

In [150]:
df_sales["DL"] = df_sales.groupby("id")["n_sales"].transform("mean")

In [151]:
df_sales["year"] = df_sales["yearweek"].astype(str).str[:4].astype(int)

In [152]:
df_sales["n_sales_sum"] = df_sales.groupby("id")["n_sales"].transform("sum")

In [153]:
df_sales["n_sales_sum_2015"] = df_sales[df_sales["year"] == 2015].groupby("id")["n_sales"].transform("sum")

In [154]:
df_sales = df_sales.dropna(subset=["n_sales_sum_2015"]) #Eliminar las filas nulas que se generan

In [155]:
df_sales["revenue"] = df_sales["n_sales"] * df_sales["avg_sell_price"]

In [156]:
df_sales["revenue_sum"] = df_sales.groupby("id")["revenue"].transform("sum")

In [157]:
df_sales["avg_price"] = df_sales["revenue_sum"] / df_sales["n_sales_sum"]

In [158]:
columnas = ["id","item","store_code","category","DL","avg_price","n_sales_sum_2015"]

In [159]:
df_sales = df_sales[columnas]

In [160]:
df_sales.shape

(1615970, 7)

In [161]:
df_sales = df_sales.drop_duplicates()

In [162]:
df_sales.shape

(30490, 7)

In [163]:
df_sales.columns

Index(['id', 'item', 'store_code', 'category', 'DL', 'avg_price',
       'n_sales_sum_2015'],
      dtype='object')

In [164]:
df_prediction.columns

Index(['id', 'rmse_mean', 'SS'], dtype='object')

In [165]:
df_final = df_sales.merge(
    df_prediction[['id','rmse_mean','SS']],  # prendi solo le colonne necessarie
    on=['id'],
    how='right'  # mantiene tutte le righe di df_prediction
)

In [166]:
df_final["ROP"] = (df_final["DL"] + df_final["SS"]).round(0)

In [167]:
df_final["SS"] = df_final["SS"].round(0)

In [168]:
df_final

,id,item,store_code,category,DL,avg_price,n_sales_sum_2015,rmse_mean,SS,ROP
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,ACCESORIES,0.948905,3.642665,87.0,1.006209,2.0,3.0
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,ACCESORIES,1.467153,5.000999,183.0,1.031386,2.0,3.0
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,ACCESORIES,1.405109,5.421564,190.0,1.477416,2.0,4.0
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,ACCESORIES,2.189781,4.119675,225.0,2.073110,3.0,6.0
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,ACCESORIES,1.978102,3.648421,180.0,2.703180,4.0,6.0
...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,SUPERMARKET,9.386861,0.397978,853.0,5.298480,9.0,18.0
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,SUPERMARKET,0.339416,0.012903,1.0,4.934147,8.0,8.0
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,SUPERMARKET,9.970803,0.575842,1311.0,9.285024,15.0,25.0
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,SUPERMARKET,3.164234,0.636678,460.0,7.789934,13.0,16.0


## 3. EOQ (Economic Order Quantity)

El **EOQ** (Economic Order Quantity) o Cantidad Económica de Pedido es un modelo de gestión de inventarios que ayuda a las empresas a decidir cuántas unidades deben pedir cada vez para mantener los costos lo más bajos posible.

El objetivo es encontrar un equilibrio entre:

- Pedir demasiado a menudo, lo que aumenta los costos de pedido, y

- Pedir grandes cantidades, lo que incrementa los costos de almacenamiento.

En resumen, el EOQ permite **optimizar la gestión de inventarios**, garantizando que haya suficiente stock para satisfacer la demanda sin generar costos innecesarios

La fórmula clásica del EOQ es:

𝐸𝑂𝑄 = √(2𝐷⋅SS/𝐻)

donde:

**D** = demanda anual (unidades por año)

**S** = costo por pedido (costo de preparación o de orden)

**H** = costo de mantenimiento por unidad al año (holding cost per unit per year)

Esta fórmula determina la cantidad óptima de pedido que minimiza el costo total de inventario, es decir, la suma de los costos de pedido y los costos de mantenimiento.

Dado que ni **H** ni **S** están disponibles para este problema, vamos a **suponer sus valores en función de la categoría de los artículos**.
En particular, **H**, el costo de mantenimiento en almacén, se calculará como un **porcentaje del precio medio de venta** del producto, de la siguiente manera:

* **SUPERMARKET:** 15 % → productos que se venden más rápido, con mayor rotación y menos tiempo en almacén.
* **HOME & GARDEN:** 20 % → productos con una rotación menor.
* **ACCESORIES:** 25 % → productos que se venden con menos frecuencia, por lo tanto, con un mayor porcentaje de costo en caso de pedido.


In [169]:
h_dict = {
    'SUPERMARKET': 0.15,
    'HOME_&_GARDEN': 0.20,
    'ACCESORIES': 0.25
}

In [170]:
df_final['H_%'] = df_final['category'].map(h_dict)

In [171]:
df_final[df_final['category']=="ACCESORIES"].head()

,id,item,store_code,category,DL,avg_price,n_sales_sum_2015,rmse_mean,SS,ROP,H_%
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,ACCESORIES,0.948905,3.642665,87.0,1.006209,2.0,3.0,0.25
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,ACCESORIES,1.467153,5.000999,183.0,1.031386,2.0,3.0,0.25
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,ACCESORIES,1.405109,5.421564,190.0,1.477416,2.0,4.0,0.25
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,ACCESORIES,2.189781,4.119675,225.0,2.073110,3.0,6.0,0.25
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,ACCESORIES,1.978102,3.648421,180.0,2.703180,4.0,6.0,0.25


In [172]:
df_final['H']=df_final['H_%']*df_final['avg_price']

In [173]:
df_final

,id,item,store_code,category,DL,avg_price,n_sales_sum_2015,rmse_mean,SS,ROP,H_%,H
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,ACCESORIES,0.948905,3.642665,87.0,1.006209,2.0,3.0,0.25,0.910666
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,ACCESORIES,1.467153,5.000999,183.0,1.031386,2.0,3.0,0.25,1.250250
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,ACCESORIES,1.405109,5.421564,190.0,1.477416,2.0,4.0,0.25,1.355391
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,ACCESORIES,2.189781,4.119675,225.0,2.073110,3.0,6.0,0.25,1.029919
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,ACCESORIES,1.978102,3.648421,180.0,2.703180,4.0,6.0,0.25,0.912105
...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,SUPERMARKET,9.386861,0.397978,853.0,5.298480,9.0,18.0,0.15,0.059697
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,SUPERMARKET,0.339416,0.012903,1.0,4.934147,8.0,8.0,0.15,0.001935
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,SUPERMARKET,9.970803,0.575842,1311.0,9.285024,15.0,25.0,0.15,0.086376
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,SUPERMARKET,3.164234,0.636678,460.0,7.789934,13.0,16.0,0.15,0.095502


https://www.ascendsoftware.com/blog/the-average-cost-of-processing-a-purchase-order-a-detailed-analysis?utm_source=chatgpt.com

https://it.scribd.com/document/459659976/2019-HCKT-Metrics-Procurement-Cost-v1?utm_source=chatgpt.com

Dado que el costo por pedido (S) no está disponible, supondremos tres escenarios posibles aplicables a todas las categorías de productos:

Escenario mínimo: 10 € por pedido → procesos muy automatizados o simples.

Escenario medio: 25 € por pedido → nivel medio de digitalización y complejidad.

Escenario alto: 40 € por pedido → procesos manuales o con mayor carga administrativa y de control.

In [174]:
df_final["S1"] = 10
df_final["S2"] = 20
df_final["S3"] = 40

In [175]:
df_final

,id,item,store_code,category,DL,avg_price,n_sales_sum_2015,rmse_mean,SS,ROP,H_%,H,S1,S2,S3
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,ACCESORIES,0.948905,3.642665,87.0,1.006209,2.0,3.0,0.25,0.910666,10,20,40
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,ACCESORIES,1.467153,5.000999,183.0,1.031386,2.0,3.0,0.25,1.250250,10,20,40
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,ACCESORIES,1.405109,5.421564,190.0,1.477416,2.0,4.0,0.25,1.355391,10,20,40
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,ACCESORIES,2.189781,4.119675,225.0,2.073110,3.0,6.0,0.25,1.029919,10,20,40
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,ACCESORIES,1.978102,3.648421,180.0,2.703180,4.0,6.0,0.25,0.912105,10,20,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,SUPERMARKET,9.386861,0.397978,853.0,5.298480,9.0,18.0,0.15,0.059697,10,20,40
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,SUPERMARKET,0.339416,0.012903,1.0,4.934147,8.0,8.0,0.15,0.001935,10,20,40
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,SUPERMARKET,9.970803,0.575842,1311.0,9.285024,15.0,25.0,0.15,0.086376,10,20,40
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,SUPERMARKET,3.164234,0.636678,460.0,7.789934,13.0,16.0,0.15,0.095502,10,20,40


In [176]:
df_final['EOQ1'] = np.round(np.sqrt((2 * df_final['DL'] * df_final['S1']) / df_final['H']))
df_final['EOQ2'] = np.round(np.sqrt((2 * df_final['DL'] * df_final['S2']) / df_final['H']))
df_final['EOQ3'] = np.round(np.sqrt((2 * df_final['DL'] * df_final['S3']) / df_final['H']))

In [177]:
df_final[df_final['rmse_mean']> 50].head(150)

,id,item,store_code,category,DL,avg_price,n_sales_sum_2015,rmse_mean,SS,ROP,H_%,H,S1,S2,S3,EOQ1,EOQ2,EOQ3


In [178]:
df_final.to_csv("df_final_SS.csv", index=False, encoding="utf-8")